In [1]:
import pandas as pd
import datetime as dt
import requests_cache

history_length = 20
apikey = '5e225d64b14d31.07865608'
symbol_df_path = 'ITOT_holdings.csv'

In [2]:
from eod_historical_data import get_eod_data
import requests
import os
import datetime as dt
import requests_cache


def get_symbol_data(symbol, history_length, apikey):
    today = dt.datetime.now().strftime('%Y-%m-%d')
    history = (dt.datetime.now() - dt.timedelta(days=365 * history_length)).strftime('%Y-%m-%d')
    df = get_eod_data(symbol, 'US', start=history, end=today, api_key=apikey)
    df.loc[:, 'symbol'] = symbol
    factor = df.Adjusted_close / df.Close
    df.loc[:, 'adj_close'] = df.Adjusted_close
    df.loc[:, 'adj_open'] = df.Open * factor
    df.loc[:, 'adj_high'] = df.High * factor
    df.loc[:, 'adj_low'] = df.Low * factor
    return df


def download_symbols(api_key, history_length, output_dir, symbol_df,
                     indices, failed_indices):
    for index in indices:
        row = symbol_df.iloc[index]
        symbol = row.symbol
        sector = row.sector
        try:            
            print('\r%d: %s' % (index, symbol))
            df = get_symbol_data(symbol, history_length, apikey)
            df.loc[:, 'sector'] = sector
            df.to_parquet('%s/%s.parquet' % (output_dir, symbol))
        except:
            print('failure occured')
            failed_indices.append(index)

            
def download_data(history_length, apikey, symbol_df_path, output_dir='data', limit=1500):
    expire_after = dt.timedelta(days=1)
    session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)
    symbol_df = pd.read_csv(symbol_df_path).dropna().iloc[:limit]
    failed_indices = []
    download_symbols(apikey, history_length, output_dir, symbol_df,
                     list(range(symbol_df.shape[0])), failed_indices)
    if len(failed_indices) != 0:
        print('retrying on failed symbols')
        failed_indices2 = []
        download_symbols(apikey, history_length, output_dir, symbol_df,
                         failed_indices, failed_indices2)

In [3]:
download_data(history_length, apikey, symbol_df_path)

0: AAPL


/Users/brianmcclanahan/opt/anaconda3/lib/python3.7/site-packages/eod_historical_data/data.py:42: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  parse_dates=[0], index_col=0)


1: MSFT
2: AMZN
3: FB
4: GOOGL
5: GOOG
6: BRKB
7: TSLA
8: JNJ
9: JPM
10: V
11: PG
12: NVDA
13: UNH
14: MA
15: HD
16: DIS
17: VZ
18: PYPL
19: CMCSA
20: ADBE
21: CRM
22: BAC
23: NFLX
24: WMT
25: PFE
26: T
27: KO
28: MRK
29: INTC
30: PEP
31: ABT
32: ABBV
33: CSCO
34: TMO
35: CVX
36: COST
37: XOM
38: NKE
39: MCD
40: QCOM
41: ACN
42: AVGO
43: MDT
44: NEE
45: HON
46: TXN
47: BMY
48: UNP
49: DHR
50: LIN
51: AMGN
52: UPS
53: PM
54: C
55: WFC
56: LLY
57: LOW
58: SBUX
59: BA
60: RTX
61: ORCL
62: IBM
63: AMT
64: AMD
65: MMM
66: NOW
67: BLK
68: CHTR
69: CAT
70: INTU
71: MS
72: LMT
73: FIS
74: GE
75: TGT
76: CVS
77: XTSLA
failure occured
78: ZM
79: ISRG
80: BKNG
81: SPGI
82: MDLZ
83: DE
84: GS
85: SQ
86: ANTM
87: AXP
88: CI
89: TMUS
90: ZTS
91: TJX
92: SYK
93: AMAT
94: GILD
95: ADP
96: MO
97: PLD
98: SCHW
99: CL
100: MU
101: UBER
102: CCI
103: CSX
104: DUK
105: FDX
106: CB
107: D
108: BDX
109: LRCX
110: TFC
111: FISV
112: SO
113: CME
114: EQIX
115: NSC
116: APD
117: ITW
118: USB
119: ATVI
120: SHW


In [4]:
import pandas as pd
import os

def combine_data(datadir='data', output_file='all_symbols.parquet'):
    files = os.listdir(datadir)
    dfs = []
    cols = ['adj_close', 'adj_open', 'adj_low', 'adj_high', 'Close', 'symbol', 'sector', 'Date', 'Volume']
    for f in files:
        dfs.append(pd.read_parquet('%s/%s' % (datadir, f)).reset_index()[cols])
    ttl_df = pd.concat(dfs, axis=0, ignore_index=True)
    ttl_df.to_parquet(output_file)
    

In [5]:
combine_data()

In [6]:
df = get_symbol_data('SPY', history_length, apikey)
df.reset_index().to_parquet('market_index.parquet')

In [7]:
import pandas as pd
all_df = pd.read_parquet('all_symbols.parquet')
sector_indices = all_df.groupby(['sector', 'Date']).agg({'adj_close': 'sum'}).reset_index()

In [8]:
sector_indices.to_parquet('sector_index.parquet')